In [ ]:
#conda install -c conda-forge cupy
#conda install -c conda-forge py-xgboost-gpu
#https://www.geeksforgeeks.org/insert-image-in-a-jupyter-notebook/

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import tftinsights

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, train_test_split

from xgboost import XGBRegressor

import xgboost as xgb
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    explained_variance_score, mean_absolute_percentage_error
)
import numpy as np


In [ ]:

# Define the model without regularization
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,
    learning_rate=0.01,
    max_depth=15,
    reg_alpha=0.0,
    reg_lambda=0.0,
    subsample =0.5,
    random_state=42
)

# Train the model
model.fit(x_train, y_train)

# Predict
y_pred = model.predict(x_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # as percentage


In [ ]:
# Output
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Explained Variance Score: {explained_var:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

print ( model.get_params() )

# X G BOOST with Hyperparmater optmization. Reguarlization

In [ ]:
dataset = pd.read_csv('csv/flattened_matches.csv')
df = dataset[dataset['queue_id']== 1100].copy() #ranked data only

column_key = list(df.keys())

column_key = [item for item in column_key if (   "trait_" not in item and "unit_" not in item  and item not in ['placement'])]
#print ( column_key )
df.drop(column_key,axis='columns',inplace=True)

column_key = list(df.keys())
column_key = [item for item in column_key if ( "_item" in item)]
df.drop(column_key,axis='columns',inplace=True)


print(list(df.keys())) #remaining columns

summonUnits = ['unit_TFT14_AnnieTibbers', 'unit_TFT14_SummonLevel2', 'unit_TFT14_SummonLevel4', 'unit_TFT14_Summon_Turret']
df.drop(summonUnits,axis='columns',inplace=True)
df = df.fillna(0).infer_objects()

In [ ]:
y = df['placement'].astype(float)
x = df.drop(columns='placement')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

# Define the parameter distributions with random sampling
param_distributions = {
    'n_estimators': randint(100, 501),           # integers between 100 and 500
    'max_depth': randint(5, 12),                 # integers between 5 and 11
    'learning_rate': uniform(0.01, 0.49),        # floats between 0.01 and 0.5
    'subsample': uniform(0.1, 0.4)               # floats between 0.1 and 0.5
}

# Initialize the XGBRegressor
xgb_reg = XGBRegressor(
    tree_method="hist",
    device="cuda"
)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_reg,
    param_distributions=param_distributions,
    n_iter=20,  # You can increase for better coverage
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit the model on training data
random_search.fit(x_train, y_train)

# Output best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best CV Score (Negative MSE):", random_search.best_score_)
best_model = random_search.best_estimator_

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

MSE: 2.119077667828744
MAE: 1.1562354335912084
R² Score: 0.5982469287540966

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': 'cuda',
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'feature_weights': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 7,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 500,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.5,
 'tree_method': 'hist',
 'validate_parameters': None,
 'verbosity': None}

In [ ]:
#with lasso now


from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

# xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
#                           colsample_bytree = 0.3, 
#                           learning_rate = 0.1,
#                           max_depth = 5, 
#                           alpha = 10, 
#                           n_estimators = 10)


# Define the parameter distributions including L1 regularization
param_distributions = {
    'colsample_bytree' :  uniform(0.1, 0.5),
    'n_estimators': randint(100, 501),
    'max_depth': randint(5, 12),
    'learning_rate': uniform(0.01, 0.49),
    'subsample': uniform(0.1, 0.4),
    'reg_alpha': uniform(0, 1.0)  # L1 regularization (Lasso)
    #'reg_lambda': uniform(0, 1.0)  # L2 regularization (Ridge)
}

# Initialize the XGBRegressor
xgb_reg = XGBRegressor(
    tree_method="hist",
    device="cuda"
)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_reg,
    param_distributions=param_distributions,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit the model on training data
random_search.fit(x_train, y_train)

# Output best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best CV Score (Negative MSE):", random_search.best_score_)
best_model = random_search.best_estimator_

In [ ]:
best_model = grid_search.best_estimator_
print ( best_model.get_params() )
y_pred = best_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

In [ ]:
best_model = grid_search.best_estimator_
print ( best_model.get_params() )
y_pred = best_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))